In [20]:
# Number of parameters for flan-t5 family: small 80M, base 250M, large 780M, xl 3B, xxl 11B
model_id = google/flan-t5-small

# https://huggingface.co/datasets/billsum
dataset_name, dataset_version = cnn_dailymail, 3.0.0

In [21]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()

2.219.0


In [4]:
!python train.py --model-name t5-small --train-dir local_train_data --valid-dir local_valid_data --output-data-dir ./output --model-dir ./model

INFO:__main__:training set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 287113
})
INFO:__main__:validation set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 11490
})
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sample

In [8]:
!python train2.py --model-name t5-small --train-dir local_train_data --valid-dir local_valid_data --output-data-dir ./output --model-dir ./model

Traceback (most recent call last):
  File "/home/sagemaker-user/styx_notebooks/notebooks/train2.py", line 89, in <module>
    model = AutoModelForSeq2SeqLM.from_pretrained(args.model_name, quantization_config=bnb_config)
  File "/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py", line 563, in from_pretrained
    return model_class.from_pretrained(
  File "/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py", line 3202, in from_pretrained
    hf_quantizer.validate_environment(
  File "/opt/conda/lib/python3.10/site-packages/transformers/quantizers/quantizer_bnb_8bit.py", line 62, in validate_environment
    raise RuntimeError("No GPU found. A GPU is needed for quantization.")
RuntimeError: No GPU found. A GPU is needed for quantization.


In [ ]:
import transformers
import datasets

print(transformers.__version__)
print(datasets.__version__)

In [ ]:
import evaluate
import rouge_score

In [ ]:
from datasets import load_dataset, load_from_disk

dataset = load_dataset(dataset_name, dataset_version)
dataset

Preprocess dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

prefix = summarize: 
input_max_length = 1024
output_max_length = 128


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples[article]]
    model_inputs = tokenizer(inputs, max_length=input_max_length, truncation=True)
    labels = tokenizer(
        text_target=examples[highlights], max_length=output_max_length, truncation=True
    )
    model_inputs[labels] = labels[input_ids]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(
    preprocess_function, batched=True, remove_columns=[article, highlights, id]
)

In [6]:
import boto3
import os

s3_client = boto3.client('s3')
bucket = styx-nlp-datasets  # Replace with your actual bucket name
s3_prefix = huggingface/cnn_dailymail-t5-summarization

dataset_input_path = s3://{}/{}.format(bucket, s3_prefix)
train_input_path = {}/train.format(dataset_input_path)
valid_input_path = {}/validation.format(dataset_input_path)

print(dataset_input_path)
print(train_input_path)
print(valid_input_path)

s3://styx-nlp-datasets/huggingface/cnn_dailymail-t5-summarization
s3://styx-nlp-datasets/huggingface/cnn_dailymail-t5-summarization/train
s3://styx-nlp-datasets/huggingface/cnn_dailymail-t5-summarization/validation


In [25]:
# Save tokenized dataset locally first
local_train_path = local_train_data
local_valid_path = local_valid_data

tokenized_dataset[train].save_to_disk(local_train_path)
tokenized_dataset[test].save_to_disk(local_valid_path)

# Upload the local files to S3
for root, dirs, files in os.walk(local_train_path):
    for file in files:
        s3_client.upload_file(
            os.path.join(root, file),
            bucket,
            os.path.join(s3_prefix, 'train', os.path.relpath(os.path.join(root, file), local_train_path))
        )

for root, dirs, files in os.walk(local_valid_path):
    for file in files:
        s3_client.upload_file(
            os.path.join(root, file),
            bucket,
            os.path.join(s3_prefix, 'validation', os.path.relpath(os.path.join(root, file), local_valid_path))
        )

# Verify by listing the uploaded files
response = s3_client.list_objects_v2(Bucket=bucket, Prefix=s3_prefix)
for obj in response.get('Contents', []):
    print(obj['Key'])


Saving the dataset (0/3 shards):   0%|          | 0/287113 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11490 [00:00<?, ? examples/s]

huggingface/cnn_dailymail-t5-summarization/train/data-00000-of-00003.arrow
huggingface/cnn_dailymail-t5-summarization/train/data-00001-of-00003.arrow
huggingface/cnn_dailymail-t5-summarization/train/data-00002-of-00003.arrow
huggingface/cnn_dailymail-t5-summarization/train/dataset_info.json
huggingface/cnn_dailymail-t5-summarization/train/state.json
huggingface/cnn_dailymail-t5-summarization/validation/data-00000-of-00001.arrow
huggingface/cnn_dailymail-t5-summarization/validation/dataset_info.json
huggingface/cnn_dailymail-t5-summarization/validation/state.json


In [7]:
train_ds = load_from_disk(train_input_path)
valid_ds = load_from_disk(valid_input_path)

NameError: name 'load_from_disk' is not defined

In [26]:
%%sh -s $dataset_input_path
aws s3 ls --recursive $1

2024-05-30 23:17:11  424111416 huggingface/cnn_dailymail-t5-summarization/train/data-00000-of-00003.arrow
2024-05-30 23:17:18  460139096 huggingface/cnn_dailymail-t5-summarization/train/data-00001-of-00003.arrow
2024-05-30 23:17:26  453302224 huggingface/cnn_dailymail-t5-summarization/train/data-00002-of-00003.arrow
2024-05-30 23:17:18       2195 huggingface/cnn_dailymail-t5-summarization/train/dataset_info.json
2024-05-30 23:17:31        368 huggingface/cnn_dailymail-t5-summarization/train/state.json
2024-05-30 23:17:32   52967328 huggingface/cnn_dailymail-t5-summarization/validation/data-00000-of-00001.arrow
2024-05-30 23:17:33       2195 huggingface/cnn_dailymail-t5-summarization/validation/dataset_info.json
2024-05-30 23:17:33        249 huggingface/cnn_dailymail-t5-summarization/validation/state.json


In [28]:
!pygmentize train.py 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


import argparse
import logging
import os

import evaluate
import numpy as np
from datasets import load_from_disk
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

rouge = evaluate.load("rouge")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}


if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    # hyperparameters are passed 

In [36]:
pip install --upgrade fsspec datasets evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
ERROR: Cannot uninstall fsspec 2023.6.0, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps fsspec==2023.6.0'.
Note: you may need to restart the kernel to use updated packages.


In [37]:
!pip show fsspec
!pip show datasets
!pip show evaluate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Exception:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3109, in _dep_map
    return self.__dep_map
  File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2902, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3100, in _parsed_pkg_info
    return self._pkg_info
  File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2902, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 180

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Exception:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3109, in _dep_map
    return self.__dep_map
  File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2902, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3100, in _parsed_pkg_info
    return self._pkg_info
  File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2902, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 180

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Exception:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3109, in _dep_map
    return self.__dep_map
  File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2902, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3100, in _parsed_pkg_info
    return self._pkg_info
  File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2902, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 180

In [29]:
hyperparameters = {
    epochs: 1,
    learning-rate: 1e-6,
    train-batch-size: 1,
    eval-batch-size: 8,
    model-name: model_id,
}

'arn:aws:iam::381491949871:role/service-role/AmazonSageMaker-ExecutionRole-20240514T182453'

In [34]:
import sagemaker
import boto3

iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName=AmazonSageMaker-ExecutionRole-20240514T182453)['Role']['Arn']
sess = sagemaker.Session()

ClientError: An error occurred (AccessDenied) when calling the GetRole operation: User: arn:aws:iam::381491949871:user/mlflow-user is not authorized to perform: iam:GetRole on resource: role AmazonSageMaker-ExecutionRole-20240514T182453 because no identity-based policy allows the iam:GetRole action

In [30]:
from sagemaker.huggingface import HuggingFace

huggingface_estimator = HuggingFace(
    role=sagemaker.get_execution_role(),
    # Fine-tuning script
    entry_point=train.py,
    dependencies=[requirements.txt],
    hyperparameters=hyperparameters,
    # Infrastructure
    transformers_version=4.26.0,
    pytorch_version=1.13.1,
    py_version=py39,
    instance_type=ml.p3dn.24xlarge,
    instance_count=1,
    distribution={smdistributed: {dataparallel: {enabled: True}}},
)

Couldn't call 'get_role' to get Role ARN from role name mlflow-user to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::381491949871:user/mlflow-user, therefore it cannot be used as a SageMaker execution role

In [23]:
from datasets.filesystems import S3FileSystem

ImportError: cannot import name 'S3FileSystem' from 'datasets.filesystems' (/opt/conda/lib/python3.10/site-packages/datasets/filesystems/__init__.py)

In [ ]:

s3 = S3FileSystem()

s3_prefix = huggingface/billsum-t5-summarization

dataset_input_path = s3://{}/{}.format(bucket, s3_prefix)
train_input_path = {}/train.format(dataset_input_path)
valid_input_path = {}/validation.format(dataset_input_path)

print(dataset_input_path)
print(train_input_path)
print(valid_input_path)

In [11]:
import boto3

In [4]:
import pandas as pd

In [7]:
def read_string_file_s3(bucket, s3_path):
    
    Read a file to an S3 bucket
    :param bucket: S3 bucket name
    :param s3_path: S3 path where the file will be stored
    
    s3 = boto3.client('s3')
    try:
        obj = s3.get_object(Bucket=bucket, Key=s3_path)
        return StringIO(obj['Body'].read().decode('utf-8'))
    except FileNotFoundError:
        print(The file was not found.)
    except NoCredentialsError:
        print(Credentials not available.)

In [8]:
file_path = external/sentiment/Fin_Cleaned.csv
bucket_name = styx-nlp-datasets

In [12]:
df_test = pd.read_csv(read_string_file_s3(bucket_name, file_path))

NameError: name 'NoCredentialsError' is not defined

In [13]:
from transformers import pipeline 

In [14]:
model_id = google/flan-t5-large 

In [15]:
summarizer = pipeline(summarization, model=model_id) 

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [22]:
summarizer("THE ECONOMIC SCENE: The Recovery Still Lives. Little wonder the stock market has been bounding up and down like a yoryo lately. There are so many anomalies in the economic picture these days and so much uncertainty over what lies aheadr-for domestic business, inflation. Government Policies; ehersv ahd taxation as well as over the whole international pic* ture—that no one seems to have much reliance on aiiy forecasts, whatever they predict.</br></br>One week, or even a single day, produces reasons for cheer, but the very next period uncovers contrary data or assessments to chase the euphoria. Investors, like businessmen, don't know what to expect next.</br></br>Although the economy seldom moves in an unbroken trend in any direction, the gyrations this year seem to have been unusually pronounced. And so have the policy actions in .Washington. Even so, over time, the economy has managed to trace a steady and strongly upward course throughout the 33 months .of the current expansionary phase of the business cycle in this country. Moreover, the conseasus is that the continuing recovery still has considerable life left in it</br></br>Despite that fact, the stock market has been unwilling to look beyond the valley to the next crest. And businessmen have not been inclined to discount the question marks of the moment to gaze toward a better tomorrow. And yet, when all the pluses and minuses are tabulated, the favorable side of the ledger is definitely longer, and the bottom line shows ai positive balance.</br></br>The last few weeks provide a good example of the economy’s erratic fluctuations and the nervous state of current thinking in so many quarters.lt should have been a consoling period for analysts of the American business scene—with many more constructive.devel-opments than negative ones—but the sense of malaise continued to linger. Inventory of Unsold Homes Grows. A new factor is adding to (lie cost of buying a newly built home in this period of inflation, high interest rates cjnd a scarcity of mortgage money—the builder's delay in selling completed homes.</br></br>were 441,000 new homes without buyers in various stages of planning or construction, according to the Census Bureau. Of the total, first half of 1974, construction of new one-family homes fell 24 per cent from the comparable 1973 period.</br></br>“It looks as if the year’s total will fall below the million mark, the first time since 1970,” said a Census Bureau official. The million mark has been exceeded only four times in the nation’s history—in 1963, 1971, 1972 and 1973.</br></br>penses. In one form or another these are eventually passed on to the buyer. The expenses include maintain-ance, security, sales operation, taxes and carrying charges on the construction loan. “The builder may cut back his profit on a project, but someplace along the line those extra costs are going to be tacked on somewhere, maybe at another development,” said a spokesman for one major builder.</br></br>including gas. The Butman construction Corporation was the contractor and Ginsbern & Associates was the architect. James Felt-Huberth & Huberth are the managing and renting agents for the $3.4-million project. Thomas Eddy was i trustee of the bank in the early 19th Century and a philanthropist. Are Large Caps Back in Style?. ®M1D rising interest rates anti U market volatility, many In-vestment strategists say it is time to refocus on those core investments: large-capitalisation domestic stock funds, We feel very strongly that the tide has gone out with respect to more-risky asset classes,” said Nick Bolm-sack, investment strategist at the ISI Group in New York. Investors should look toward higher-quality asset classes, and in our view the most attractive is U.S. large caps.</br></br>Despite their usually prominent position in investor portfolios, large-capitalization domestic stocks and stock funds, which include Standard Despite the lackluster performance Of large-cap slocks, their earnings and price-to-earnings ratios have ' become more attractive.</br></br>& Poor’s 500-stock index funds, have been out of favor with investors in recent years, and no wonder. Domestic small-cap stock funds and emerging-market funds outperformed domestic large caps for the three years through April 2006, posting gains of better than 20 percent a year. The</br></br>Investors actually pulled a net $10.2 billion out of large-cap blend funds in 2005 and early 2006, according to the Financial Research Corporation. That was the first time investors took money out of this group of core stock funds since 1994, when the company began tracking fund categories. Large-cap blend funds meld growth and value characteristics in a single portfolio.</br></br>“Market history is highly cyclical: winners become losers and vice versa,” said Francis M. Kinniry Jr., a principal in the investment counseling and research group at Vanguard. “Investors would do themselves a huge service by going back to core funds,” he said.")

Token indices sequence length is longer than the specified maximum sequence length for this model (1256 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'Little wonder the stock market has been bounding up and down like a yory'}]